In [43]:
import pandas as pd
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, RobustScaler
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score

# Carga y Exploración Inicial de Datos

## Carga del Dataset

Se procede a cargar el dataset desde el archivo CSV ubicado en la carpeta `data/`. Este archivo contiene los datos necesarios para la Etapa 1 del proyecto.

## Análisis Estadístico Descriptivo

Se utiliza la función `describe()` para obtener un resumen estadístico de las variables numéricas presentes en el dataset. Esta función proporciona métricas clave como:

- **count**: Cantidad de valores no nulos en cada columna
- **mean**: Promedio de los valores
- **std**: Desviación estándar (dispersión de los datos)
- **min/max**: Valores mínimo y máximo
- **25%, 50%, 75%**: Percentiles que indican la distribución de los datos

Este análisis permite identificar rápidamente patrones, valores atípicos y la distribución general de las variables numéricas del dataset.

In [44]:
data_raw = pd.read_csv("../data/Datos_Etapa-1.csv")
data_raw.describe()
data_raw.head()

,season,weekday,weathersit,temp,atemp,hum,windspeed,cnt,time_of_day
0,Winter,6,Clear,3.28,3.0014,0.81,0.0,16,Night
1,Winter,6,Clear,2.34,1.9982,0.80,0.0,40,Night
2,Winter,6,Clear,2.34,1.9982,0.80,0.0,32,Night
3,Winter,6,Clear,3.28,3.0014,0.75,0.0,13,Night
4,Winter,6,Clear,3.28,3.0014,0.75,0.0,1,Night


## Identificación de Valores Faltantes

Se analiza la presencia de valores nulos en el dataset mediante `isna().sum()`, que cuenta la cantidad de valores faltantes por columna.

Este paso permite identificar qué variables tienen datos incompletos y determinar la estrategia de limpieza apropiada (imputación o eliminación).

In [45]:
data_raw.isna().sum()

season         0
weekday        0
weathersit     0
temp           0
atemp          0
hum            0
windspeed      0
cnt            0
time_of_day    0
dtype: int64

## Detección de Registros Duplicados

Se verifica la cantidad de filas duplicadas en el dataset usando `duplicated().sum()`. Esta función identifica registros que son idénticos en todas sus columnas.

Detectar duplicados es esencial para asegurar la calidad de los datos y evitar sesgos en el análisis posterior.

In [46]:
data_raw.duplicated().sum()

np.int64(42)

## Eliminación de Duplicados

Se procede a eliminar todas las filas duplicadas del dataset usando `drop_duplicates()`. Posteriormente, se verifica con `duplicated().sum()` que la eliminación fue exitosa.

Este proceso asegura que cada registro sea único, evitando redundancia y posibles distorsiones en el análisis de datos.

In [47]:
data = data_raw.drop_duplicates()
data.duplicated().sum()

np.int64(0)

## Codificación de Variables Categóricas

Se aplica **One-Hot Encoding** a las variables categóricas (`season`, `weathersit`, `time_of_day`), transformándolas en variables binarias (0 o 1).

In [48]:
data_encoded = data.copy()
data_encoded = pd.get_dummies(data_encoded, columns=['season', 'weathersit', 'time_of_day'], drop_first=True)
data_encoded.head()

,weekday,temp,atemp,hum,windspeed,cnt,season_Spring,season_Summer,season_Winter,weathersit_Heavy Rain,weathersit_Light Rain,weathersit_Mist,time_of_day_Morning,time_of_day_Night
0,6,3.28,3.0014,0.81,0.0,16,False,False,True,False,False,False,False,True
1,6,2.34,1.9982,0.80,0.0,40,False,False,True,False,False,False,False,True
2,6,2.34,1.9982,0.80,0.0,32,False,False,True,False,False,False,False,True
3,6,3.28,3.0014,0.75,0.0,13,False,False,True,False,False,False,False,True
4,6,3.28,3.0014,0.75,0.0,1,False,False,True,False,False,False,False,True


## Preparación de Datos para Entrenamiento

Se separan las variables predictoras (features) de la variable objetivo:

- **X**: Contiene todas las variables independientes (features) excepto `cnt`
- **y**: Contiene la variable objetivo `cnt` (demanda de bicicletas)

Posteriormente, se divide el dataset en conjuntos de entrenamiento y prueba usando `train_test_split`:

- **80%** de los datos para entrenamiento (`x_train`, `y_train`)
- **20%** de los datos para prueba (`x_test`, `y_test`)
- `random_state=77` asegura reproducibilidad de los resultados

Esta división permite evaluar el rendimiento del modelo en datos no vistos durante el entrenamiento.

In [49]:
x = data_encoded.drop(['cnt'], axis="columns")
y = data_encoded['cnt']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=77)

## Creación de Features Polinomiales (Grado 2)

Se generan características polinomiales de grado 2 mediante `PolynomialFeatures`. Esto permite capturar relaciones no lineales entre variables, creando términos cuadráticos e interacciones entre features.

Posteriormente, se normaliza el dataset transformado usando **RobustScaler** para mantener todas las variables en una escala comparable y preservar la estabilidad numérica del modelo.

In [50]:
pol_2 = PolynomialFeatures(degree=2)
x_train_pol2 = pol_2.fit_transform(x_train)

pol_2_scaler = RobustScaler()
x_train_pol2 = pol_2_scaler.fit_transform(x_train_pol2)

## Entrenamiento del Modelo de Regresión Polinomial (Grado 2)

Se entrena un modelo de **Regresión Lineal** utilizando las características polinomiales de grado 2. El modelo aprende los pesos óptimos que minimizan el error entre las predicciones y los valores reales de demanda en el conjunto de entrenamiento.

In [51]:
reg_polinomial2 = LinearRegression().fit(x_train_pol2, y_train)

## Transformación del Conjunto de Prueba

Se aplican las mismas transformaciones al conjunto de prueba:

- Se generan características polinomiales de grado 2 usando el transformador ajustado previamente
- Se normaliza con el escalador ya entrenado

In [52]:
x_test_pol2 = pol_2.transform(x_test)
x_test_pol2 = pol_2_scaler.transform(x_test_pol2)

## Evaluación del Modelo Polinomial (Grado 2)

Se realizan predicciones sobre el conjunto de prueba y se evalúa el rendimiento del modelo mediante tres métricas:

- **RMSE** (Root Mean Squared Error): Mide el error promedio de las predicciones, penalizando más los errores grandes
- **MAE** (Mean Absolute Error): Representa el error absoluto promedio en las mismas unidades que la variable objetivo
- **R²** (Coeficiente de determinación): Indica qué porcentaje de la variabilidad de la demanda es explicado por el modelo (valores cercanos a 1 indican mejor ajuste)

In [53]:
y_pred_pol2 = reg_polinomial2.predict(x_test_pol2)

print(f'------ Modelo de regresión polinomial grado 2 ----')
print(f"RMSE: {root_mean_squared_error(y_test, y_pred_pol2):.2f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred_pol2):.2f}")
print(f'R²: {r2_score(y_test, y_pred_pol2):.2f}')


------ Modelo de regresión polinomial grado 2 ----
RMSE: 135.96
MAE: 99.08
R²: 0.45


## Creación de Features Polinomiales (Grado 3)

Se generan características polinomiales de grado 3 mediante `PolynomialFeatures`. Esto permite capturar relaciones no lineales más complejas entre variables, creando términos cúbicos e interacciones de mayor orden.

Posteriormente, se normaliza el dataset transformado usando **RobustScaler** para mantener todas las variables en una escala comparable y preservar la estabilidad numérica del modelo.

In [54]:
pol_3 = PolynomialFeatures(degree=3)
x_train_pol3 = pol_3.fit_transform(x_train)

pol_3_scaler = RobustScaler()
x_train_pol3 = pol_3_scaler.fit_transform(x_train_pol3)

## Entrenamiento del Modelo de Regresión Polinomial (Grado 3)

Se entrena un modelo de **Regresión Lineal** utilizando las características polinomiales de grado 3. El modelo aprende los pesos óptimos que minimizan el error entre las predicciones y los valores reales de demanda en el conjunto de entrenamiento.

In [55]:
reg_polinomial3 = LinearRegression().fit(x_train_pol3, y_train)

## Transformación del Conjunto de Prueba

Se aplican las mismas transformaciones al conjunto de prueba:

- Se generan características polinomiales de grado 3 usando el transformador ajustado previamente
- Se normaliza con el escalador ya entrenado

In [56]:
x_test_pol3 = pol_3.transform(x_test)
x_test_pol3 = pol_3_scaler.transform(x_test_pol3)

## Evaluación del Modelo Polinomial (Grado 3)

Se realizan predicciones sobre el conjunto de prueba y se evalúa el rendimiento del modelo mediante tres métricas:

- **RMSE** (Root Mean Squared Error): Mide el error promedio de las predicciones, penalizando más los errores grandes
- **MAE** (Mean Absolute Error): Representa el error absoluto promedio en las mismas unidades que la variable objetivo
- **R²** (Coeficiente de determinación): Indica qué porcentaje de la variabilidad de la demanda es explicado por el modelo (valores cercanos a 1 indican mejor ajuste)

In [57]:
y_pred_pol3 = reg_polinomial3.predict(x_test_pol3)

print(f'------ Modelo de regresión polinomial grado 3 ----')
print(f"RMSE: {root_mean_squared_error(y_test, y_pred_pol3):.2f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred_pol3):.2f}")
print(f'R²: {r2_score(y_test, y_pred_pol3):.2f}')

------ Modelo de regresión polinomial grado 3 ----
RMSE: 132.31
MAE: 96.12
R²: 0.48


## Selección del Modelo Óptimo

Se selecciona el **modelo de grado 3** como el más adecuado para predecir la demanda de bicicletas.

**Justificación:**

El modelo de grado 3 presenta un **RMSE de 132.31**, menor que el de grado 2 (RMSE: 135.96). Esta mejora indica que:

- Realiza predicciones más precisas
- Captura mejor los patrones en los datos
- Ofrece un buen equilibrio entre rendimiento y complejidad

El modelo de grado 3 se utiliza como modelo final por su mejor desempeño predictivo.

## Preparación de Datos para Modelo Lasso

Se preparan los datos específicamente para el modelo de regresión Lasso:

- Se separan las variables predictoras (todas excepto `cnt`) de la variable objetivo
- Se dividen los datos en conjuntos de entrenamiento (80%) y prueba (20%)
- Se utiliza `random_state=77` para garantizar reproducibilidad
- Los datos utilizados son los originales codificados, sin transformación polinomial

In [58]:
lasso_scaler = StandardScaler()
x_lasso = data_encoded.drop(['cnt'], axis="columns")
y_lasso = data_encoded['cnt']
x_train_lasso, x_test_lasso, y_train_lasso, y_test_lasso = train_test_split(x_lasso, y_lasso, test_size=0.2, random_state=77)

## Configuración del Modelo Lasso

Se configura la validación cruzada y el modelo de regresión Lasso:

- **KFold**: Se divide el conjunto de entrenamiento en 10 particiones (folds) para validación cruzada
- `shuffle=True` aleatoriza los datos antes de dividirlos
- `random_state=77` garantiza reproducibilidad
- Se inicializa el modelo **Lasso** que será optimizado mediante GridSearchCV

In [59]:
lasso_columns = x_lasso.columns
x_train_lasso = lasso_scaler.fit_transform(x_train_lasso)
x_train_lasso = pd.DataFrame(x_train_lasso, columns=lasso_columns)
kfold = KFold(n_splits=10, shuffle=True, random_state=77)
lasso = Lasso()

## Definición de Hiperparámetros a Evaluar

Se define el espacio de búsqueda para el hiperparámetro de regularización α (alpha):

- Se evalúan los valores α = [1, 2, 3, 4, 5]
- `param_grid` estructura los parámetros en formato compatible con GridSearchCV
- Cada valor de α representa un nivel diferente de penalización sobre los coeficientes del modelo

In [60]:
a_options = [1, 2, 3, 4, 5]
param_grid = {'alpha': a_options}

## Búsqueda de Hiperparámetro Óptimo con GridSearchCV

Se ejecuta **GridSearchCV** para encontrar el mejor valor de α mediante validación cruzada:

- El modelo Lasso se entrena con cada valor de α en `param_grid`
- Para cada α, se evalúa el rendimiento usando los 10 folds definidos en KFold
- Se selecciona automáticamente el α que minimiza el error de validación cruzada
- `n_jobs=1` utiliza un solo núcleo de procesamiento

In [61]:
grid = GridSearchCV(lasso, param_grid, cv=kfold, n_jobs=1)
grid.fit(x_train_lasso, y_train_lasso)

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Lasso()
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'alpha': [1, 2, ...]}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",None
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",KFold(n_split... shuffle=True)
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and parameter candidate is displayed;- >2 : the score is also displayed;- >3 : the fold and candidate paramet

## Preparación del Conjunto de Prueba

Se extrae el mejor modelo seleccionado por GridSearchCV y se prepara el conjunto de prueba:

- `best_estimator_` obtiene el modelo Lasso entrenado con el α óptimo
- Se normaliza el conjunto de prueba usando el mismo escalador (debe ser `transform`, no `fit_transform`)
- Se preservan los nombres de las columnas para mantener la coherencia con el conjunto de entrenamiento

In [62]:
mejor_modelo = grid.best_estimator_
print(f'Mejor modelo: {grid.best_params_}')
x_test_lasso = lasso_scaler.fit_transform(x_test_lasso)
x_test_lasso = pd.DataFrame(x_test_lasso, columns=lasso_columns)

Mejor modelo: {'alpha': 1}


## Identificación de Variables Relevantes

Se extraen los coeficientes del modelo Lasso para identificar qué variables fueron seleccionadas como relevantes para la predicción.

Las variables con coeficiente igual a 0 fueron eliminadas automáticamente por el modelo debido a la regularización L1, indicando que no aportan información significativa. Las variables con coeficiente diferente de 0 son las que el modelo considera importantes para predecir la demanda de bicicletas.

In [63]:
coeficientes = pd.DataFrame({
    'Variable': lasso_columns,
    'Coeficiente': mejor_modelo.coef_
})
variables_seleccionadas = coeficientes[coeficientes['Coeficiente'] != 0]
print(variables_seleccionadas)

                 Variable  Coeficiente
0                 weekday     3.235413
1                    temp    37.480726
2                   atemp    22.692611
3                     hum   -28.145973
5           season_Spring   -11.157808
6           season_Summer   -23.058952
7           season_Winter   -23.643207
9   weathersit_Light Rain   -10.998606
10        weathersit_Mist    -0.374366
11    time_of_day_Morning   -39.594441
12      time_of_day_Night   -89.831430


## Evaluación del Modelo Lasso

Se realizan predicciones sobre el conjunto de prueba y se evalúa el rendimiento del modelo con el α óptimo:

- **RMSE** (Root Mean Squared Error): Mide el error promedio de las predicciones, penalizando más los errores grandes
- **MAE** (Mean Absolute Error): Representa el error absoluto promedio en las mismas unidades que la variable objetivo
- **R²** (Coeficiente de determinación): Indica qué porcentaje de la variabilidad de la demanda es explicado por el modelo (valores cercanos a 1 indican mejor ajuste)

In [64]:
y_pred = mejor_modelo.predict(x_test_lasso)
print(f'------ Modelo de regresión Lasso ----')
print(f"RMSE: {root_mean_squared_error(y_test_lasso, y_pred):.2f}")
print(f"MAE: {mean_absolute_error(y_test_lasso, y_pred):.2f}")
print(f'R²: {r2_score(y_test_lasso, y_pred):.2f}')

------ Modelo de regresión Lasso ----
RMSE: 139.77
MAE: 103.70
R²: 0.42


# Análisis de Resultados

## 1. ¿Cuál es el grado de la transformación polinomial que fue seleccionado utilizando la técnica de validación?

El **grado 3** fue seleccionado como óptimo para la transformación polinomial.

**Justificación:**
- Grado 2: RMSE = 135.96
- Grado 3: RMSE = 132.31 

El modelo de grado 3 mostró un error menor, indicando que captura mejor las relaciones no lineales entre las variables y la demanda de bicicletas.

---

## 2. ¿Cuál fue el valor de α que fue seleccionado utilizando la técnica de validación para la regresión Lasso?

El valor **α = 1** fue seleccionado mediante GridSearchCV con validación cruzada.

Este valor de regularización proporciona el mejor balance entre ajuste del modelo y penalización de coeficientes, minimizando el error de validación cruzada.

---

## 3. A partir de la tabla comparativa, ¿Cuál modelo ofrece el mejor rendimiento sobre el conjunto test? ¿Qué interpretación puedes darles a los valores obtenidos sobre las métricas de rendimiento?

### Tabla Comparativa

| Modelo | RMSE | MAE | R² |
|--------|------|-----|-----|
| Regresión Polinomial (grado 3) | 132.31 | 96.12 | 0.48 |
| Regresión Lasso (α = 1) | 139.77 | 103.70 | 0.42 |

### Modelo con Mejor Rendimiento

El **modelo de regresión polinomial de grado 3** ofrece el mejor rendimiento en el conjunto de prueba, con un RMSE 7.46 unidades menor que el modelo Lasso.

### Interpretación de las Métricas

**Modelo Polinomial (grado 3):**
- **RMSE = 132.31**: En promedio, las predicciones del modelo se desvían aproximadamente 132 bicicletas de la demanda real. Este error es moderado considerando la variabilidad del sistema.

**Modelo Lasso (α = 1):**
- **RMSE = 139.77**: Error promedio ligeramente superior al polinomial, indicando predicciones menos precisas.
- **MAE = 103.70**: En promedio, el modelo se equivoca por 103 bicicletas en valor absoluto. Este valor es menor que el RMSE, lo que sugiere que no hay muchos errores extremos.
- **R² = 0.42**: El modelo explica solo el 42% de la variabilidad en la demanda. Esto indica que hay factores adicionales no capturados por las variables disponibles, o que las relaciones son más complejas de lo que Lasso puede modelar linealmente.

**Conclusión:** El modelo polinomial captura mejor las relaciones no lineales complejas entre las variables, mientras que Lasso, al ser más simple y regularizado, sacrifica precisión pero ofrece mayor interpretabilidad.

---

## 4. ¿Cuáles variables fueron seleccionadas con el modelo Lasso? A partir de estas, ¿Qué interpretación de cara al problema puedes dar? Reflexiona sobre cómo este nuevo conocimiento podría ayudar a tomar decisiones en el contexto del problema.

El modelo Lasso seleccionó **11 variables** como relevantes para predecir la demanda:


| Variable | Coeficiente | Interpretación |
|----------|-------------|----------------|
| time_of_day_Night | -89.83 | Fuerte reducción de demanda nocturna |
| time_of_day_Morning | -39.59 | Reducción moderada en la mañana |
| temp | +37.48 | Mayor temperatura aumenta demanda |
| hum | -28.15 | Mayor humedad reduce demanda |
| season_Winter | -23.64 | Invierno reduce demanda |
| season_Summer | -23.06 | Verano reduce demanda |
| atemp | +22.69 | Sensación térmica positiva aumenta demanda |
| season_Spring | -11.16 | Primavera reduce levemente demanda |
| weathersit_Light Rain | -11.00 | Lluvia ligera reduce demanda |
| weekday | +3.24 | Efecto leve del día de la semana |
| weathersit_Mist | -0.37 | Efecto mínimo de neblina |

### Interpretación del Problema

**Factores Temporales:**
- El **momento del día** es el factor más determinante. La demanda cae drásticamente en la noche (-89.8) y moderadamente en la mañana (-39.6), sugiriendo que el uso principal ocurre en tarde/noche temprana.

**Factores Climáticos:**
- La **temperatura** tiene un impacto positivo significativo (+37.5), indicando que las personas prefieren usar bicicletas con clima cálido y agradable.
- La **humedad** (-28.1) y la **lluvia ligera** (-11.0) desalientan el uso, lo cual es intuitivo.

**Factores Estacionales:**
- Todas las estaciones muestran coeficientes negativos, sugiriendo que la estación de referencia (probablemente Otoño) tiene la mayor demanda base.
- **Invierno** y **Verano** tienen impactos similares y negativos, posiblemente por temperaturas extremas.

**Variables Eliminadas:**
- Lasso eliminó `windspeed` (velocidad del viento), sugiriendo que no es un factor determinante o está correlacionado con otras variables.

### Aplicaciones para Toma de Decisiones

Este conocimiento permite:

1. **Optimización de Recursos:**
   - Aumentar disponibilidad de bicicletas en tardes/noches y reducirla durante la noche
   - Ajustar inventario según temperatura prevista y condiciones climáticas

2. **Estrategias de Negocio:**
   - Implementar descuentos o promociones en días de alta humedad o lluvia ligera para mantener demanda
   - Planificar mantenimiento preventivo durante la noche cuando la demanda es mínima

3. **Planificación Estacional:**
   - Preparar mayor capacidad en otoño (estación de referencia con mayor demanda)
   - Reducir operaciones en invierno y verano extremos

4. **Pronóstico de Demanda:**
   - Integrar pronósticos meteorológicos para anticipar fluctuaciones de demanda
   - Ajustar precios dinámicos según condiciones que favorecen o reducen uso

**Valor del Modelo Lasso:** Aunque menos preciso que el polinomial, Lasso identifica claramente qué variables importan y en qué magnitud, facilitando decisiones operativas concretas para gestores del sistema de bicicletas compartidas.


---

## 👥 Créditos

**Autor:** David Cortes M
**Institución:** Universidad de los Andes
**Programa:** Maestría en Inteligencia Artificial  
**Curso:** Principios de Machine Learning
**Fecha:** Febrero 2026

---

## 🔗 Repositorio del Proyecto

El código completo y los datos de este proyecto están disponibles en:

**GitHub:** [https://github.com/dacors-m/ml-principles](https://github.com/dacors-m/ml-principles)

---

## 📧 Contacto

Para consultas o colaboraciones:
- GitHub: [@dacors-m](https://github.com/dacors-m)
- Email: [dacors.m@gmail.com]

---

*Proyecto desarrollado como parte de la Etapa 1 del curso de Machine Learning - Predicción de Demanda en Sistemas de Alquiler de Bicicletas*